In [7]:
#https://deepsense.ai/satellite-images-semantic-segmentation-with-deep-learning/
import torch
import torchvision

In [8]:
#Convolution Neural Network
class ConvRelu(torch.nn.Module):
    def __init__(self, in_depth, out_depth):
      super(ConvRelu, self).__init__()
      self.conv = torch.nn.Conv2d(in_depth, out_depth, kernel_size=3, stride=1, padding=1)
      self.activation = torch.nn.ReLU(inplace=True)
    def forward(self, x):
      x = self.conv(x)
      x = self.activation(x)
      return x

In [9]:
#Decode Block - Upsizing tensor to generate images from Neural Network
class DecoderBlock(torch.nn.Module):
    def __init__(self, in_depth, middle_depth, out_depth):
      super(DecoderBlock, self).__init__()
      self.conv_relu = ConvRelu(in_depth, middle_depth)
      self.conv_transpose = torch.nn.ConvTranspose2d(middle_depth, out_depth, kernel_size=4, stride=2, padding=1)
      self.activation = torch.nn.ReLU(inplace=True)
    def forward(self, x):
      x = self.conv_relu(x)
      x = self.conv_transpose(x)
      x = self.activation(x)
      return x

In [10]:
#UNetArchitechture
class UNetResNet(torch.nn.Module):
    def __init__(self, num_classes):
      super(UNetResNet, self).__init__()
      
      self.encoder = torchvision.models.resnet101(pretrained=True)
      
      self.pool = torch.nn.MaxPool2d(2, 2)
      self.conv1 = torch.nn.Sequential(self.encoder.conv1, self.encoder.bn1, self.encoder.relu, self.pool)
      self.conv2 = self.encoder.layer1
      self.conv3 = self.encoder.layer2
      self.conv4 = self.encoder.layer3
      self.conv5 = self.encoder.layer4
      
      self.pool = torch.nn.MaxPool2d(2, 2)      
      self.center = DecoderBlock(2048, 512, 256)
      
      self.dec5 = DecoderBlock(2048 + 256, 512, 256)
      self.dec4 = DecoderBlock(1024 + 256, 512, 256)
      self.dec3 = DecoderBlock(512 + 256, 256, 64)
      self.dec2 = DecoderBlock(256 + 64, 128, 128)
      self.dec1 = DecoderBlock(128, 128, 32)
      self.dec0 = ConvRelu(32, 32)
      self.final = torch.nn.Conv2d(32, num_classes, kernel_size=1)
    def forward(self, x):
      conv1 = self.conv1(x)
      conv2 = self.conv2(conv1)
      conv3 = self.conv3(conv2)
      conv4 = self.conv4(conv3)
      conv5 = self.conv5(conv4)
      pool = self.pool(conv5)
      center = self.center(pool)
      dec5 = self.dec5(torch.cat([center, conv5], 1))
      dec4 = self.dec4(torch.cat([dec5, conv4], 1))
      dec3 = self.dec3(torch.cat([dec4, conv3], 1))
      dec2 = self.dec2(torch.cat([dec3, conv2], 1))
      dec1 = self.dec1(dec2)
      dec0 = self.dec0(dec1)
      return self.final(dec0)

In [12]:
#Training
unet_resnet = UNetResNet(num_classes=2)
unet_resnet.train()
cross_entropy_loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(unet_resnet.parameters(), lr=0.0001, weight_decay=0.0001)

for epoch_idx in range(2):
    loss_batches = []
    for batch_idx, data in enumerate(train_dataloader):
      imgs, masks = data
      imgs = torch.autograd.Variable(imgs)
      masks = torch.autograd.Variable(masks)
      y = unet_resnet(imgs)
      loss = cross_entropy_loss(y, masks)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_batches.append(loss.data.cpu().numpy())
    print('epoch: ',str(epoch_idx),' training loss: ',str(np.sum(loss_batches)))

#Saving Model
model_file = './unet-' + str(epoch_idx)
unet_resnet = unet_resnet.cpu()
torch.save(unet_resnet.state_dict(), model_file)
unet_resnet = unet_resnet.cuda()
print('model saved')

NameError: name 'train_dataloader' is not defined

In [ ]:
#Prediction
unet_resnet = UNetResNet(num_classes=2)
model_path= './unet-99'
pretrained_model = torch.load(model_path)
for name, tensor in pretrained_model.items():
    unet_resnet.state_dict()[name].copy_(tensor)
unet_resnet.eval()
softmax2d = torch.nn.Softmax2d()
img = cv2.imread('./img.png')
assert img.shape[0] % 64 == 0 and img.shape[1] % 64 == 0
img = np.expand_dims(img, axis=0)
img = (img / 255.0 - MEAN) / STD
img = img.transpose(0, 3, 1, 2)
img = torch.FloatTensor(img)
img = img.cuda()
with torch.no_grad():
    pred = unet_resnet(img)
    pred = softmax2d(pred)
    pred = pred[0, 1, :, :] > 0.7
pred = pred.data.cpu().numpy()
mask = (pred * 255.0).astype(np.uint8)
cv2.imwrite('./mask.png', mask)